# Risk-Neutral Valuation and Numerical Methods

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Load the Libs we need

In [1]:
# import Lib
import pandas as pd
import datetime as dt
import pytz
import os
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import scipy.stats as si
import math
import networkx as nx

# import module
from datetime import datetime, timezone
from datetime import date, time
from math import trunc
from dateutil.parser import parse

## Implementing numerical methods for options pricing in Python

#### Constructing the Crank-Nicolson Scheme

In [8]:
import numpy as np
from scipy.linalg import solve

# Parameters
S_max = 300
K = 145
r = 0.05
sigma = 0.2
T = 1
N_S = 100
N_T = 1000
dt = T / N_T
ds = S_max / N_S

# Set up A and B matrices
A = np.zeros((N_S - 2, N_S - 2))
B = np.zeros((N_S - 2, N_S - 2))

for i in range(1, N_S - 1):
    j = i - 1
    A[j, j] = 1 + dt * (sigma**2 * i**2 + r)
    B[j, j] = 1 - dt * (sigma**2 * i**2 + r)
    if i > 1:
        A[j, j - 1] = -0.5 * dt * (sigma**2 * i**2 - r * i)
        B[j, j - 1] = 0.5 * dt * (sigma**2 * i**2 - r * i)
    if i < N_S - 2:
        A[j, j + 1] = -0.5 * dt * (sigma**2 * i**2 + r * i)
        B[j, j + 1] = 0.5 * dt * (sigma**2 * i**2 + r * i)

# Initial condition: payoff at maturity for a call option
V_call = np.maximum(np.arange(0, S_max, ds) - K, 0)[1:-1]

# Initial condition: payoff at maturity for a put option
V_put = np.maximum(K - np.arange(0, S_max, ds), 0)[1:-1]

# Time-stepping loop for the call option
for t in range(N_T):
    b_call = B @ V_call
    # Adjust for boundary conditions if necessary
    V_call = solve(A, b_call)

# Time-stepping loop for the put option
for t in range(N_T):
    b_put = B @ V_put
    # Adjust for boundary conditions if necessary
    V_put = solve(A, b_put)

# Option prices at S = $150
call_option_price = V_call[int(150 / ds) - 1]
put_option_price = V_put[int(150 / ds) - 1]

print(f"The European call option price is: ${call_option_price:.2f}")
print(f"The European put option price is: ${put_option_price:.2f}")


The European call option price is: $23.24
The European put option price is: $8.14
